# Calculate post-TT dM/M0 and CBF time series for every ROI

Calculate the perfusion (dM/M0) time series  (i.e. vs PLD) for each ROI and filter only PLDs that are longer than the arterial transit time. Then calculate CBF using the white paper model, substituting assumed vs calculated T1 of arterial blood (aka T1a) parameter values. The best T1blood value should provide a flat response for PLD>ATT. That is because once the label reaches the voxel (PLD>TT) it just sits there and relaxes with T1b, so adjusting the CBF calculation for PLD with the correct T1a (arterial blood T1) should result in the same CBF regardless of PLD.


In [1]:
import os
import sys
import pandas as pd
import numpy as np
import subprocess
import glob
import flywheel

local_dir = '../data/subjects'

In [2]:
def refine_list(lst):
    '''
    creates a usable list of floats from the fsl output string

    :param lst: fsl output string
    :return: list of floats
    
    '''
    result_list = str(lst).split(' ')
    result_list[0] = result_list[0].replace("b'", "")
    result_list = result_list[: len(result_list) - 1] # remove extra text
    result_list = [float(e) for e in result_list]
    
    return(result_list)
    

In [3]:
cols = ['session_id', 'gray_matter','left_acc', 'right_acc', 'left_insula', 'right_insula', 'left_caudate', 'right_caudate', 'left_putamen', 'right_putamen']
df = pd.DataFrame(columns=cols)
for root, dirs, files in os.walk(local_dir, topdown=False):
    pw_maps = [f for f in files if 'dMM0' in f]
    if len(pw_maps) > 0:
        for pw_map in pw_maps:
            ses_name = pw_map.split('dMM0')[0][:-1] # drop the last underscore
            pw_map_path = os.path.join(root, pw_map)
            roi_dir = os.path.join(root, ses_name + "_rois")
            
            for mask in glob.glob(roi_dir+"/*roimask.nii.gz"):
                cmd1="fslmeants --transpose -i '{}' -m '{}'".format(pw_map_path, mask)
                result = refine_list(subprocess.check_output(cmd1, shell=True))
                if 'left_acc' in mask:
                    left_acc = result
                elif 'right_acc' in mask:
                    right_acc = result
                elif 'left_insula' in mask:
                    left_insula = result
                elif 'right_insula' in mask:
                    right_insula = result
                elif 'left_caudate' in mask:
                    left_caudate = result
                elif 'right_caudate' in mask:
                    right_caudate = result
                elif 'left_putamen' in mask:
                    left_putamen = result
                elif 'right_putamen' in mask:
                    right_putamen = result
                elif 'GM' in mask:
                    gray_matter = result

            
            
            df.loc[len(df.index)] = [ses_name, gray_matter, left_acc, right_acc, left_insula, right_insula, left_caudate, right_caudate, left_putamen, right_putamen]

df.head()

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen
0,sub-HC010_ses-MR_FOLLOWUP,"[0.013107, 0.018295, 0.01631, 0.015468, 0.0112...","[0.003617, 0.009817, 0.010409, 0.009573, 0.012...","[0.02497, 0.029967, 0.020654, 0.015629, 0.0163...","[0.045235, 0.030914, 0.017905, 0.016068, 0.008...","[0.017875, 0.017218, 0.013523, 0.012861, 0.008...","[0.005339, 0.00824, 0.006975, 0.006414, 0.0080...","[0.007093, 0.016956, 0.015067, 0.015163, 0.012...","[0.015342, 0.015347, 0.012845, 0.011406, 0.007...","[0.006176, 0.014828, 0.012481, 0.010933, 0.008..."
1,sub-HC010_ses-MR_BASELINE,"[0.008968, 0.01323, 0.013456, 0.010283, 0.0108...","[-0.002361, 0.00646, 0.009186, 0.007178, 0.005...","[0.009802, 0.021089, 0.018035, 0.010873, 0.008...","[0.0342, 0.022725, 0.01461, 0.009205, 0.010548...","[0.015618, 0.011586, 0.009735, 0.007871, 0.008...","[0.002249, 0.004813, 0.004249, 0.002536, 0.003...","[0.008587, 0.010943, 0.006538, 0.008575, 0.009...","[0.010816, 0.010513, 0.009618, 0.007115, 0.007...","[0.004655, 0.010256, 0.008434, 0.007559, 0.006..."
2,sub-HC017_ses-MR_BASELINE,"[0.028187, 0.033478, 0.0273, 0.022279, 0.01828...","[0.013306, 0.018888, 0.018058, 0.013668, 0.012...","[0.048275, 0.041003, 0.027638, 0.021547, 0.017...","[0.041497, 0.035631, 0.0217, 0.021234, 0.01458...","[0.023444, 0.020573, 0.014642, 0.011624, 0.010...","[0.007886, 0.010077, 0.009671, 0.006364, 0.007...","[0.026201, 0.021915, 0.014684, 0.011715, 0.009...","[0.025358, 0.023975, 0.016818, 0.014622, 0.011...","[0.021776, 0.019861, 0.015653, 0.008639, 0.008..."
3,sub-HC017_ses-MR_FOLLOWUP,"[0.021711, 0.022417, 0.02225, 0.017671, 0.0156...","[0.009382, 0.013484, 0.014594, 0.00855, 0.0080...","[0.037073, 0.030628, 0.024858, 0.015602, 0.013...","[0.039901, 0.027992, 0.020475, 0.017916, 0.015...","[0.019759, 0.016196, 0.013668, 0.009976, 0.010...","[0.006356, 0.010461, 0.008747, 0.004586, 0.004...","[0.016478, 0.015816, 0.014733, 0.009868, 0.008...","[0.01989, 0.017704, 0.015066, 0.011941, 0.0103...","[0.017751, 0.016715, 0.013663, 0.007375, 0.007..."
4,sub-HC028_ses-MR_BASELINE,"[0.01203, 0.017401, 0.015652, 0.010334, 0.0121...","[0.005034, 0.00993, 0.009799, 0.008688, 0.0111...","[0.034014, 0.03541, 0.021585, 0.009503, 0.0138...","[0.031228, 0.023756, 0.017826, 0.010339, 0.010...","[0.019991, 0.017738, 0.011007, 0.006186, 0.008...","[0.003093, 0.007013, 0.006702, 0.00503, 0.0058...","[0.012094, 0.016889, 0.011188, 0.005648, 0.009...","[0.013403, 0.016448, 0.013737, 0.007457, 0.009...","[0.009118, 0.018809, 0.010998, 0.005188, 0.010..."


In [4]:
# example
list(df.loc[df['session_id']=='sub-HC001_ses-MR_BASELINE', 'left_acc'])[0]

[0.005529, 0.011766, 0.014897, 0.011584, 0.011633, 0.011795, 0.006563]

## Select time points based on transit time for that region
Take the time series list from the above calculation and exclude time points (i.e. PLDs) that happen before ATT (i.e. when all the blood has arrived). The PLDs in this experiment are [1, 1.22, 1.48, 1.78, 2.15,x, y]. We can determine which to keep based on how long the transit-time was using a decision tree. E.g. if the transit time was between the first (1s) and second (1.22s) delays, then we eliminate the first from the data. We do this by adding a NaN at the end, so that the resulting time series is PLDs after ATT (i.e. first PLD after, second PLD after) rather than the absolute (first (1s), second (1.22s), etc.)

In [5]:
# Check the maximum transit-time
tt_df = pd.read_csv("regionals_tts2.csv")
np.max(tt_df)

session_id       sub-HC028_ses-MR_FOLLOWUP
gray_matter                    1793.280349
left_acc                       1885.564396
right_acc                      1752.977169
left_insula                    1533.890533
right_insula                   1404.397273
left_caudate                   1524.748148
right_caudate                  1959.006284
left_putamen                   1624.468193
right_putamen                   1951.27004
dtype: object

It looks like the maximum transit time in the dataset is not more than the fifth PLD (2.15s). So there should be at least 3 points in every time series.

In [6]:
refined_df = df.copy()

for row, row in df.iterrows(): # iterate over rows
    row_list = [row.session_id] 
    ses_id = row.session_id
    for c, value in row.items():
        if type(value) == list:
            tt = float(tt_df.loc[tt_df['session_id']==row.session_id, c])/1000 # originally in ms, but we want s
            
            # extract post-TT values
            if tt < 1:
                dms = value[0:7] # all 7
            elif 1 < tt < 1.22:
                dms = value[1:7] + [np.nan]
            elif 1.22 < tt < 1.48:
                dms = value[2:7] + [np.nan]*2
            elif 1.48 < tt < 1.78:
                dms = value[3:7] + [np.nan]*3
            elif 1.78 < tt < 2.15:
                dms = value[4:7] + [np.nan]*4 # 5,6,7 see above comment
            else:
                print("??")

            row_list.append(dms)

    refined_df.loc[len(refined_df.index)] = row_list
    
refined_df = refined_df.tail(50) # only get the newly created rows
refined_df = refined_df.reset_index(drop=True)
refined_df.head()

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen
0,sub-HC010_ses-MR_FOLLOWUP,"[0.015468, 0.011229, 0.010575, 0.007563, nan, ...","[0.010409, 0.009573, 0.012502, 0.008696, 0.004...","[0.020654, 0.015629, 0.016335, 0.012376, 0.006...","[0.017905, 0.016068, 0.008584, 0.013125, 0.007...","[0.017218, 0.013523, 0.012861, 0.008912, 0.008...","[0.00824, 0.006975, 0.006414, 0.008052, 0.0064...","[0.015163, 0.012297, 0.009674, 0.007837, nan, ...","[0.012845, 0.011406, 0.007149, 0.008084, 0.006...","[0.010933, 0.008899, 0.005981, 0.005354, nan, ..."
1,sub-HC010_ses-MR_BASELINE,"[0.010283, 0.010845, 0.008505, 0.007467, nan, ...","[0.009186, 0.007178, 0.005835, 0.005082, 0.004...","[0.010873, 0.008415, 0.005807, 0.006227, nan, ...","[0.01461, 0.009205, 0.010548, 0.006135, 0.0030...","[0.009735, 0.007871, 0.008109, 0.007217, 0.005...","[0.004813, 0.004249, 0.002536, 0.00381, 0.0036...","[0.009588, 0.008429, 0.008493, nan, nan, nan, ...","[0.009618, 0.007115, 0.007522, 0.00637, 0.0050...","[0.006073, 0.005443, 0.00439, nan, nan, nan, nan]"
2,sub-HC017_ses-MR_BASELINE,"[0.0273, 0.022279, 0.018281, 0.016482, 0.01230...","[0.018888, 0.018058, 0.013668, 0.012188, 0.010...","[0.041003, 0.027638, 0.021547, 0.017655, 0.018...","[0.035631, 0.0217, 0.021234, 0.014584, 0.01665...","[0.020573, 0.014642, 0.011624, 0.010119, 0.008...","[0.010077, 0.009671, 0.006364, 0.00741, 0.0045...","[0.014684, 0.011715, 0.009306, 0.008801, 0.006...","[0.023975, 0.016818, 0.014622, 0.011355, 0.011...","[0.015653, 0.008639, 0.008741, 0.005968, 0.005..."
3,sub-HC017_ses-MR_FOLLOWUP,"[0.02225, 0.017671, 0.01564, 0.013513, 0.01302...","[0.013484, 0.014594, 0.00855, 0.008028, 0.0091...","[0.024858, 0.015602, 0.01389, 0.014549, 0.0135...","[0.027992, 0.020475, 0.017916, 0.015815, 0.013...","[0.016196, 0.013668, 0.009976, 0.010179, 0.006...","[0.010461, 0.008747, 0.004586, 0.00446, 0.0044...","[0.014733, 0.009868, 0.008488, 0.006118, 0.007...","[0.017704, 0.015066, 0.011941, 0.010377, 0.008...","[0.013663, 0.007375, 0.007584, 0.006265, 0.004..."
4,sub-HC028_ses-MR_BASELINE,"[0.010334, 0.012185, 0.010931, 0.009333, nan, ...","[0.009799, 0.008688, 0.011168, 0.009733, 0.006...","[0.009503, 0.013822, 0.011599, 0.009188, nan, ...","[0.017826, 0.010339, 0.010303, 0.008395, 0.007...","[0.011007, 0.006186, 0.008663, 0.007371, 0.005...","[0.006702, 0.00503, 0.005877, 0.005118, 0.0042...","[0.005648, 0.009298, 0.008564, 0.006218, nan, ...","[0.013737, 0.007457, 0.009954, 0.007661, 0.007...","[0.005188, 0.010818, 0.008003, 0.005842, nan, ..."


## Pull Hematocrit data from Flywheel
The hematocrit data is essential for calculating T1a. It lives on Flywheel in each subject's custom info. 

In [7]:
fw = flywheel.Client()
project_id = '5e50277c6dea314fa72a7440'
project = fw.get(project_id)

# initialize target columns with NaNs
refined_df['Hb'] = np.nan
refined_df['T1a'] = np.nan

sessions = [fw.get_session(x.id) for x in fw.get_project_sessions(project_id)]

for ses in sessions:
    subject = fw.get(ses.parents['subject'])
    session_id = "sub-{}_ses-{}".format(subject.label, ses.label) # NOTE this is not the Flywheel definition of 'id'

    if subject.info:
        hb = subject.info['Hb']
        t1a = hb / 3 #??  conversion equation
        refined_df.loc[refined_df['session_id']==session_id, 'Hb'] = hb
        refined_df.loc[refined_df['session_id']==session_id, 'T1a'] = t1a 

refined_df.to_csv('refined_dM2.csv', index=False)
refined_df.head()

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen,Hb,T1a
0,sub-HC010_ses-MR_FOLLOWUP,"[0.015468, 0.011229, 0.010575, 0.007563, nan, ...","[0.010409, 0.009573, 0.012502, 0.008696, 0.004...","[0.020654, 0.015629, 0.016335, 0.012376, 0.006...","[0.017905, 0.016068, 0.008584, 0.013125, 0.007...","[0.017218, 0.013523, 0.012861, 0.008912, 0.008...","[0.00824, 0.006975, 0.006414, 0.008052, 0.0064...","[0.015163, 0.012297, 0.009674, 0.007837, nan, ...","[0.012845, 0.011406, 0.007149, 0.008084, 0.006...","[0.010933, 0.008899, 0.005981, 0.005354, nan, ...",16.9,5.633333
1,sub-HC010_ses-MR_BASELINE,"[0.010283, 0.010845, 0.008505, 0.007467, nan, ...","[0.009186, 0.007178, 0.005835, 0.005082, 0.004...","[0.010873, 0.008415, 0.005807, 0.006227, nan, ...","[0.01461, 0.009205, 0.010548, 0.006135, 0.0030...","[0.009735, 0.007871, 0.008109, 0.007217, 0.005...","[0.004813, 0.004249, 0.002536, 0.00381, 0.0036...","[0.009588, 0.008429, 0.008493, nan, nan, nan, ...","[0.009618, 0.007115, 0.007522, 0.00637, 0.0050...","[0.006073, 0.005443, 0.00439, nan, nan, nan, nan]",16.9,5.633333
2,sub-HC017_ses-MR_BASELINE,"[0.0273, 0.022279, 0.018281, 0.016482, 0.01230...","[0.018888, 0.018058, 0.013668, 0.012188, 0.010...","[0.041003, 0.027638, 0.021547, 0.017655, 0.018...","[0.035631, 0.0217, 0.021234, 0.014584, 0.01665...","[0.020573, 0.014642, 0.011624, 0.010119, 0.008...","[0.010077, 0.009671, 0.006364, 0.00741, 0.0045...","[0.014684, 0.011715, 0.009306, 0.008801, 0.006...","[0.023975, 0.016818, 0.014622, 0.011355, 0.011...","[0.015653, 0.008639, 0.008741, 0.005968, 0.005...",14.5,4.833333
3,sub-HC017_ses-MR_FOLLOWUP,"[0.02225, 0.017671, 0.01564, 0.013513, 0.01302...","[0.013484, 0.014594, 0.00855, 0.008028, 0.0091...","[0.024858, 0.015602, 0.01389, 0.014549, 0.0135...","[0.027992, 0.020475, 0.017916, 0.015815, 0.013...","[0.016196, 0.013668, 0.009976, 0.010179, 0.006...","[0.010461, 0.008747, 0.004586, 0.00446, 0.0044...","[0.014733, 0.009868, 0.008488, 0.006118, 0.007...","[0.017704, 0.015066, 0.011941, 0.010377, 0.008...","[0.013663, 0.007375, 0.007584, 0.006265, 0.004...",14.5,4.833333
4,sub-HC028_ses-MR_BASELINE,"[0.010334, 0.012185, 0.010931, 0.009333, nan, ...","[0.009799, 0.008688, 0.011168, 0.009733, 0.006...","[0.009503, 0.013822, 0.011599, 0.009188, nan, ...","[0.017826, 0.010339, 0.010303, 0.008395, 0.007...","[0.011007, 0.006186, 0.008663, 0.007371, 0.005...","[0.006702, 0.00503, 0.005877, 0.005118, 0.0042...","[0.005648, 0.009298, 0.008564, 0.006218, nan, ...","[0.013737, 0.007457, 0.009954, 0.007661, 0.007...","[0.005188, 0.010818, 0.008003, 0.005842, nan, ...",15.5,5.166667


## Calculate CBF


In [8]:
def calc_cbf_ts(dMM0_list, T1blood):
    '''
    calculates CBF values from list of DM/M0 values

    :param dMM0_list:  the list of dM/M0 values
    :param T1blood: the assumed (1.65) or calculated T1 relaxation time of blood
    :return: list of CBF values
    
    '''
    
    plds = [1, 1.22, 1.48, 1.78, 2.15, 2.63, 3.32] # post-labeling durations
    cbf_ts = []
    
    # set the parameters
    lmbda = 0.9
    alpha = 0.85
    tau = 3.5 # labeling duration
    
    denominator = 2*alpha*T1blood*(1-np.exp(-tau/T1blood)) # only need to calculate once                         
                                   
    for i in range(len(dMM0_list)-1):
        # calculate CBF factor based on corresponding PLD value
        pld = plds[i]                                   
        numerator = 6000 * lmbda * np.exp(pld/T1blood)
        factor = numerator/denominator                           
        # multiply by deltaM over M0 to get the CBF value for this PLD                          
        cbf = dMM0_list[i] * factor
        cbf_ts.append(cbf) 
                                
    return cbf_ts
    

In [9]:
cbf_df = refined_df.copy()
for row, row in refined_df.iterrows(): #iterate over rows
    row_list = [row.session_id]
    for c, value in row.items():
        if type(value) == list:
            cbf_ts = calc_cbf_ts(value, 1.65) 
            row_list.append(cbf_ts)
    
    
    row_list.append(row.Hb)
    row_list.append(row.T1a)
#     print(row_list)
    cbf_df.loc[len(cbf_df.index)] = row_list

cbf_df = cbf_df.tail(50) # only get the newly created rows
cbf_df = cbf_df.reset_index(drop=True)
# cbf_df.to_csv('cbf_ts.csv', index=False)
cbf_df.head()

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen,Hb,T1a
0,sub-HC010_ses-MR_FOLLOWUP,"[62.02474901761784, 51.44910981964697, 56.7219...","[41.7387905691999, 43.86163757266724, 67.05799...","[82.81996161170667, 71.60906023432742, 87.6173...","[71.79681478927124, 73.62047346888303, 46.0426...","[69.04203055245306, 61.95977487675537, 68.9835...","[33.04137134116699, 31.95810321418093, 34.4032...","[60.801737092975124, 56.34247960211941, 51.889...","[51.50684646569053, 52.26008964314662, 38.3456...","[43.83996515448771, 40.7734997136912, 32.08077...",16.9,5.633333
1,sub-HC010_ses-MR_BASELINE,"[41.23354629869176, 49.689695965274865, 45.618...","[36.83471324514077, 32.888210017403686, 31.297...","[43.59937264472193, 38.55590516807635, 31.1474...","[58.58427612796719, 42.17553262889397, 56.5771...","[39.03613470949765, 36.06340220771585, 43.4949...","[19.299529158378242, 19.46809757090391, 13.602...","[38.44668306057149, 38.62005046485034, 45.5545...","[38.56697931545438, 32.59955618192076, 40.3463...","[24.351971863459603, 24.938775024342192, 23.54...",16.9,5.633333
2,sub-HC017_ses-MR_BASELINE,"[109.46959194342946, 102.07807620196945, 98.05...","[75.73852207426724, 82.73826922461352, 73.3121...","[164.41691129877063, 126.63197944566774, 115.5...","[142.8758619244079, 99.42520999967401, 113.894...","[82.49516172352287, 67.08681681176161, 62.3485...","[40.40751201516259, 44.310654650085134, 34.135...","[58.881006890011655, 53.675867979086675, 49.91...","[96.13675702724254, 77.05682865320357, 78.4292...","[62.76657592272899, 39.58222991646008, 46.8848...",14.5,4.833333
3,sub-HC017_ses-MR_FOLLOWUP,"[89.2197223714764, 80.96510994950411, 83.88954...","[54.069156694696076, 66.86689007996509, 45.860...","[99.67747679596226, 71.48535144769188, 74.5029...","[112.24442555606144, 93.81249653195047, 96.097...","[64.94393813610928, 62.62413687905734, 53.5090...","[41.947304077663574, 40.07706506300223, 24.598...","[59.07749077298704, 45.21327061183332, 45.5277...","[70.99082988155587, 69.0295029426308, 64.04891...","[54.78692434883065, 33.790825979151876, 40.678...",14.5,4.833333
4,sub-HC028_ses-MR_BASELINE,"[41.43804993199267, 55.82931722792755, 58.6314...","[39.29276671991448, 39.80673845516902, 59.9027...","[38.105843671736636, 63.32973514357115, 62.214...","[71.48003465141296, 47.37130166758661, 55.2630...","[44.13669591653216, 28.34305756027573, 46.4664...","[26.87418334083752, 23.046488769509686, 31.522...","[22.647774919285336, 42.60164067174972, 45.935...","[55.08365511087511, 34.16653414597092, 53.3910...","[20.80323234441436, 49.565987178639325, 42.926...",15.5,5.166667


### Refine CBF ts data based on transit time, like for dM/M0 data
Again, take the time series list from the above calculation and exclude time points (i.e. PLDs) that happen before ATT (i.e. when all the blood has arrived).

In [10]:
tt_df = pd.read_csv("regionals_tts2.csv")
refined_cbf_df = cbf_df.copy()

for row, row in cbf_df.iterrows(): #iterate over rows
    row_list = [row.session_id]
    for c, value in row.items():
        if type(value) == list:
            
            tt = float(tt_df.loc[tt_df['session_id']==row.session_id, c])/1000
 
            # extract post-TT values
            if tt < 1:
                dms = value[0:7]
            elif 1 < tt < 1.22:
                dms = value[1:7] + [0]
            elif 1.22 < tt < 1.48:
                dms = value[2:7] + [0]*2
            elif 1.48 < tt < 1.78:
                dms = value[3:7] + [0]*3
            elif 1.78 < tt < 2.15:
                dms = value[4:7] + [0]*4
            else:
                print("??")

            row_list.append(dms)
            
    row_list.append(row.Hb)
    row_list.append(row.T1a)
    refined_cbf_df.loc[len(refined_cbf_df.index)] = row_list
    
refined_cbf_df = refined_cbf_df.tail(50) # only get the newly created rows
refined_cbf_df.reset_index(drop=True)
refined_cbf_df.to_csv('refined_cbf.csv', index=False)